# Data Pipeline

## Part 1: Webscraping news articles

### Install packages

In [4]:
import html5lib
import bs4
import requests
from bs4 import BeautifulSoup

### Access contents

In [ ]:
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9"}
URLS = ["https://www.vox.com/donald-trump/401174/trump-fired-generals", "https://www.foxnews.com/world/trump-says-minerals-deal-has-been-pretty-much-negotiated-zelenskyy-meeting-slated-friday"]
for url in URLS:
    r = requests.get(url, headers=headers)
    print(r.content)

### Parse contents

In [ ]:
soup = BeautifulSoup(r.content, 'html5lib')
print(soup.prettify())

In [9]:
headline_text=[]
table = soup.find('script')
for row in table.find_all('script', attrs = {'headline':'articleBody'}):
    headline = row.find('headline').text.strip()
    text = row.find('articleBody').text.strip()
    title_text.append({'headline': headline, 'text':text})

In [10]:
headline_text

[]

## Part 2: Uploading and parsing subjective intensifiers data

#### Import dependencies

In [13]:
import numpy as np
import pandas as pd

#### Read in data

In [24]:
#https://stackoverflow.com/questions/64966365/parse-sgml-using-beautiful-soup
with open("acl-99-data/subjectivity.data", "r") as f:
    entries = f.read()
    soup = BeautifulSoup(entries, "html.parser")
print(soup)

<txt art="FEAT" docnum="891101-0108" tnum="??">
<mc cluster="2" cmt="rs" soa="other" uid="1">
<wf pos="DT" quote="0" stem="an">An</wf>
<mc cluster="2" cmt="" soa="JANADDEDTHISLINE" uid="2">
<wf pos="CC" quote="0" stem="but">but</wf>
<s s_id="891101-0108_0_1" snum="1">
<mc cluster="2" cmt="rs" soa="other" uid="3">
<wf nc="pronoun" pos="PRP" quote="0" stem="it">It</wf>
<s s_id="891101-0108_0_2" snum="2">
<mc cluster="2" soa="other" uid="4">
<wf nc="noun" pos="NNP" quote="0" stem="amateur">Amateur</wf>
<p pnum="1">
<s s_id="891101-0108_1_0" snum="0">
<mc cluster="1" soa="other" uid="5">
<wf nc="pronoun" pos="PRP" quote="0" stem="he">He</wf>
<mc cluster="2" cmt="" soa="JANADDEDTHISLINE" uid="6">
<wf pos="DT" quote="0" stem="an">an</wf>
<mc cluster="2" cmt="" soa="JANADDEDTHISLINE" uid="7">
<wf pos="CC" quote="0" stem="and">and</wf>
<s s_id="891101-0108_1_1" snum="1">
<mc1 cluster="2" soa="se.ms" uid="8">
<wf nc="pronoun" pos="PRP" quote="0" stem="he">He</wf>
<mc1 cluster="2" soa="se.ms" ui

#### Basic assessment of data

In [35]:
entries=[]
table = soup.find('txt')
for row in table.find_all_next('mc'):
    entries_dict = {}
    if 'cluster' in row.mc:
        entries_dict['subj'] = row.mc['cluster']
    if 'cmt' in row.mc:
        entries_dict['cmt'] = row.mc['cmt']
    if 'soa' in row.mc:
        entries_dict['soa'] = row.mc['soa']
    if 'uid' in row.mc:
        entries_dict['uid'] = row.mc['uid']
    if 'nc' in row.wf:
        entries_dict['nc'] = row.wf['nc']
    if 'pos' in row.wf:
        entries_dict['pos'] = row.wf['pos']
    if 'quote' in row.wf:
        entries_dict['quote'] = row.wf['quote']
    if 'stem' in row.wf:
        entries_dict['stem'] = row.wf['stem']
    if 'pnum' in row.p:
        entries_dict['pnum'] = row.p['pnum']
    if 's_id' in row.s:
        entries_dict['s_id'] = row.s['s_id']
    if 'snum' in row.s:
        entries_dict['snum'] = row.s['snum']
    entries.append(entries_dict)

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.

TypeError: argument of type 'NoneType' is not iterable